# KASPI BANK TASK 1 SOLUTION

Importing all libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
import torch.autograd
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim

In [2]:
file_name = 'task2.xlsx'
xl = pd.ExcelFile(file_name)
print(xl.sheet_names)

[u'\u0443\u0441\u043b\u043e\u0432\u0438\u044f', u'train', u'test']


In [3]:
TRAIN_SHEET = xl.parse(u'train')
TEST_SHEET = xl.parse(u'test')

In [4]:
print(TRAIN_SHEET.shape)
print(TEST_SHEET.shape)

(30000, 132)
(10000, 131)


In [5]:
trainLength = len(TRAIN_SHEET['NUM'])
testLength = len(TEST_SHEET['NUM'])
print(trainLength)
print(testLength)

30000
10000


In [6]:
TEST_SHEET['TARGET'] = pd.Series(np.random.randn(testLength), index=TEST_SHEET.index)

In [7]:
MAIN_DATA = pd.concat([TRAIN_SHEET, TEST_SHEET], ignore_index=True)
MAIN_DATA[MAIN_DATA.select_dtypes(['object']).columns] = MAIN_DATA.select_dtypes(['object']).apply(lambda x: x.astype('category'))
cat_columns = MAIN_DATA.select_dtypes(['category']).columns
MAIN_DATA[cat_columns] = MAIN_DATA[cat_columns].apply(lambda x: x.cat.codes)
DATA = MAIN_DATA.fillna(MAIN_DATA.mean())

In [8]:
scaler = MinMaxScaler()
scaler.fit(DATA.drop('TARGET', axis = True).drop('NUM', axis = True))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [9]:
TRAIN_VAL_DATA = DATA[:trainLength]
TEST_DATA = (DATA[trainLength:]).drop('NUM', axis=True).drop('TARGET', axis=True)
TEST_NORMALIZED_DATA = scaler.transform(TEST_DATA)

In [10]:
X = TRAIN_VAL_DATA.drop('NUM', axis = True).drop('TARGET', axis = True)
Y = TRAIN_VAL_DATA['TARGET']

In [11]:
new_Y = []
for item in Y:
    tmp = []
    if(item == 1.0):
        tmp = [0.0 , 1.0]
    else:
        tmp = [1.0 , 0.0]
    new_Y.append(tmp)

In [12]:
new_Y = np.array(new_Y)

In [13]:
X_norm = scaler.transform(X)

In [14]:
X_tensor = torch.FloatTensor(X_norm)
Y_tensor = torch.FloatTensor(new_Y)

In [15]:
X_batches = X_tensor.split(40)
Y_batches = Y_tensor.split(40)

In [16]:
class Classifier(nn.Module):
    def __init__(self, nlabel):
        super(Classifier, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(130, 256),
            nn.ReLU(),
            nn.Linear(256, 2),
        )

    def forward(self, input):
        return self.main(input)
classifier = Classifier(2)

In [28]:
optimizer = optim.Adam(classifier.parameters())
optimizer = optim.SGD(classifier.parameters(), lr=1e-4)
criterion = nn.MultiLabelSoftMarginLoss()

epochs = 10000
for epoch in range(epochs):
    losses = []
    for labels, samples in zip(Y_batches, X_batches):
        input_datas = Variable(samples)
        input_labels = Variable(labels)
        output = classifier(input_datas)
        loss = criterion(output, input_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean())
    if epoch % 1000==0:
        print('[%d/%d] Loss: %.3f' % (epoch+1, epochs, np.mean(losses)))

[1/10000] Loss: 0.030
[1001/10000] Loss: 0.005
[2001/10000] Loss: 0.005
[3001/10000] Loss: 0.005
[4001/10000] Loss: 0.004
[5001/10000] Loss: 0.004
[6001/10000] Loss: 0.004
[7001/10000] Loss: 0.004
[8001/10000] Loss: 0.004
[9001/10000] Loss: 0.004


In [29]:
print(output)
print(input_labels)

Variable containing:
 -14.8018   15.4912
 236.8300 -251.1372
 -27.5599   29.7573
 195.6243 -210.0303
-156.9685  168.4965
  14.6241  -16.0722
  26.4513  -27.6635
  21.4839  -23.1648
 162.4092 -172.4937
  25.0294  -25.5903
-102.0571  109.8498
 -27.9657   28.9878
 -21.6309   23.0183
 -25.6304   26.5020
  75.8059  -82.1391
  13.6559  -15.8104
 181.1954 -192.9361
  66.7367  -71.5015
 131.6038 -142.3121
 116.4446 -125.0342
   5.9431   -6.2896
  35.3527  -38.4012
  22.7954  -24.4426
  95.3763 -101.9027
 210.4176 -222.5711
 144.9043 -156.4185
-256.6062  272.3921
 -18.5455   19.3113
 537.3000 -565.9764
 -12.3588   14.3003
 315.2156 -340.3169
 207.9001 -220.6423
  48.4231  -51.7094
  76.3449  -81.5526
  51.4271  -54.6342
 144.5528 -158.2618
 180.0813 -192.7925
-240.9494  257.8958
  14.1040  -11.4442
 -14.2782   16.1731
[torch.FloatTensor of size 40x2]

Variable containing:
    0     1
    1     0
    0     1
    1     0
    0     1
    1     0
    1     0
    1     0
    1     0
    1     0
    

In [31]:
torch.save(classifier.state_dict(), './classifier-SGD-10000.pth')

In [17]:
X_test_tensor =  torch.FloatTensor(TEST_NORMALIZED_DATA)

In [19]:
classifier = Classifier(2)
classifier.load_state_dict(torch.load('./classifier-SGD-10000.pth'))
classifier.eval()
input_test_datas = Variable(X_test_tensor)
predict = classifier(input_test_datas).data.numpy()

In [20]:
predict
answer = []
for item, num in zip(predict, range(30000, 40000)):
    tmp = []
    if(item[0] > item[1]):
        tmp = [num, 0]
    else:
        tmp = [num, 1]
    answer.append(tmp)
answer = np.array(answer)

In [21]:
print(type(answer))

<type 'numpy.ndarray'>


In [23]:
df = pd.DataFrame (answer, columns=['NUM','TARGET'])

filepath = 'output-2.xlsx'

df.to_excel(filepath, index=False)